In [ ]:
import pandas as pd

# List of CSV file names
csv_files = [
    'Electrical_permits_part_1.csv',
    'Electrical_permits_part_2.csv',
    'Electrical_permits_part_3.csv',
    'Electrical_permits_part_4.csv',
    'Electrical_permits_part_5.csv',
    'Electrical_permits_part_6.csv',
    'Electrical_permits_part_7.csv',
    'Electrical_permits_part_8.csv'
]

# Read and concatenate all the CSV files into one DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('Electrical_permits.csv', index=False)

# Optionally, display the first few rows of the merged DataFrame
print(merged_df.head())


In [ ]:
import pandas as pd

df = pd.read_csv('Electrical_permits.csv')

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load your dataset
df = pd.read_csv('Electrical_permits.csv')

# Strip any leading or trailing spaces in column names
df.columns = df.columns.str.strip()

# Print the column names to ensure they are correct
print("Original columns:", df.columns)

# Check for the presence of the predictor columns
if 'PermitClassMapped' not in df.columns or 'PermitTypeMapped' not in df.columns or 'PermitClass' not in df.columns:
    raise KeyError("The columns 'PermitClass', 'PermitClassMapped', or 'PermitTypeMapped' are not found in the DataFrame")

# Extract rows with and without missing values
non_missing_data = df.dropna(subset=['EstProjectCost'])
missing_data = df[df['EstProjectCost'].isnull()]

# Convert categorical variables to numerical using get_dummies
non_missing_data = pd.get_dummies(non_missing_data, columns=['PermitClass', 'PermitClassMapped', 'PermitTypeMapped'])
missing_data = pd.get_dummies(missing_data, columns=['PermitClass', 'PermitClassMapped', 'PermitTypeMapped'])

# Ensure both datasets have the same dummy variable columns
missing_data = missing_data.reindex(columns=non_missing_data.columns, fill_value=0)

# Print the columns after get_dummies to ensure they match
print("Non-missing data columns after get_dummies:", non_missing_data.columns)
print("Missing data columns after reindex:", missing_data.columns)

# Choose predictors - update this list based on the actual dummy columns created
predictors = [col for col in non_missing_data.columns if col.startswith('PermitClass_') or col.startswith('PermitClassMapped_') or col.startswith('PermitTypeMapped_')]

# Train the regression model
model = LinearRegression()
model.fit(non_missing_data[predictors], non_missing_data['EstProjectCost'])

# Predict the missing values
predicted_values = model.predict(missing_data[predictors])

# Fill in the missing values with the predicted values
df.loc[df['EstProjectCost'].isnull(), 'EstProjectCost'] = predicted_values

# Verify if the missing values are filled
print(df['EstProjectCost'].isnull().sum())


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load your dataset
df = pd.read_csv('Electrical_permits.csv')

# Strip any leading or trailing spaces in column names
df.columns = df.columns.str.strip()

# Print the column names to ensure they are correct
print("Original columns:", df.columns)

# Check for the presence of the predictor columns
if 'PermitClassMapped' not in df.columns or 'PermitTypeMapped' not in df.columns or 'PermitClass' not in df.columns:
    raise KeyError("The columns 'PermitClass', 'PermitClassMapped', or 'PermitTypeMapped' are not found in the DataFrame")

# Extract rows with and without missing values in EstProjectCost
non_missing_data = df.dropna(subset=['EstProjectCost'])
missing_data = df[df['EstProjectCost'].isnull()]

# Convert categorical variables to numerical using get_dummies
non_missing_data = pd.get_dummies(non_missing_data, columns=['PermitClass', 'PermitClassMapped', 'PermitTypeMapped'])
missing_data = pd.get_dummies(missing_data, columns=['PermitClass', 'PermitClassMapped', 'PermitTypeMapped'])

# Ensure both datasets have the same dummy variable columns
missing_data = missing_data.reindex(columns=non_missing_data.columns, fill_value=0)

# Print the columns after get_dummies to ensure they match
print("Non-missing data columns after get_dummies:", non_missing_data.columns)
print("Missing data columns after reindex:", missing_data.columns)

# Choose predictors - update this list based on the actual dummy columns created
predictors = [col for col in non_missing_data.columns if col.startswith('PermitClass_') or col.startswith('PermitClassMapped_') or col.startswith('PermitTypeMapped_')]

# Train the regression model
model = LinearRegression()
model.fit(non_missing_data[predictors], non_missing_data['EstProjectCost'])

# Predict the missing values
predicted_values = model.predict(missing_data[predictors])

# Fill in the missing values with the predicted values
df.loc[df['EstProjectCost'].isnull(), 'EstProjectCost'] = predicted_values

# Fill other missing values using backfill or forward fill
df.fillna(method='bfill', inplace=True)
df.fillna(method='ffill', inplace=True)

# Define thresholds
single_family_threshold = 15000000  # 15 million
multi_family_threshold = 50000000   # 50 million

# Check unique values in 'PermitClass' to ensure categories are correct
print("Unique values in 'PermitClass':", df['PermitClass'].unique())

# Print descriptive statistics for 'EstProjectCost'
print("Descriptive statistics for 'EstProjectCost':")
print(df['EstProjectCost'].describe())

# Print the number of rows before filtering
print("Number of rows before filtering:", len(df))

# Filter rows based on the conditions
filtered_df = df[
    ~(
        ((df['PermitClass'] == 'Single Family/Duplex') & (df['EstProjectCost'] > single_family_threshold)) |
        ((df['PermitClass'] == 'MultiFamily') & (df['EstProjectCost'] > multi_family_threshold))
    )
]

# Print the number of rows after filtering
print("Number of rows after filtering:", len(filtered_df))

# Verify if the missing values are filled
print(filtered_df.isnull().sum())

# Save the cleaned dataset to a new CSV file
filtered_df.to_csv('Cleaned_Electrical_permits55.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
print(filtered_df.head(5))


In [ ]:


# Convert date columns to datetime with error handling
date_columns = ['AppliedDate', 'IssuedDate', 'ExpiresDate', 'CompletedDate']
for col in date_columns:
    filtered_df[col] = pd.to_datetime(filtered_df[col], errors='coerce')

# Verify if the datetime conversion was successful
print(filtered_df[date_columns].dtypes)




In [ ]:
filtered_df['ApplicationToIssueTime'] = (filtered_df['IssuedDate'] - filtered_df['AppliedDate']).dt.days
filtered_df['IssueToCompletionTime'] = (filtered_df['CompletedDate'] - filtered_df['IssuedDate']).dt.days
filtered_df['TotalPermitTime'] = (filtered_df['CompletedDate'] - filtered_df['AppliedDate']).dt.days
filtered_df['AppliedYear'] = filtered_df['AppliedDate'].dt.year
filtered_df['AppliedMonth'] = filtered_df['AppliedDate'].dt.month
filtered_df['AppliedDay'] = filtered_df['AppliedDate'].dt.day


# Data Analysis

In [ ]:
# Heatmap of Permits by Year and Month
filtered_df['Year'] = filtered_df['AppliedDate'].dt.year
filtered_df['Month'] = filtered_df['AppliedDate'].dt.month
heatmap_data = filtered_df.pivot_table(index='Year', columns='Month', aggfunc='size', fill_value=0)
fig = px.imshow(heatmap_data, labels={'x': 'Month', 'y': 'Year', 'color': 'Number of Permits'}, title='Heatmap of Permits by Year and Month')
fig.show()


In [ ]:
# Box Plot of Estimated Project Costs by Permit Class
fig = px.box(filtered_df, x='PermitClass', y='EstProjectCost', labels={'PermitClass': 'Permit Class', 'EstProjectCost': 'Estimated Project Cost'}, title='Box Plot of Estimated Project Costs by Permit Class')
fig.show()



In [ ]:
# Ensure only numeric columns are used for correlation matrix
numeric_df = filtered_df.select_dtypes(include=[np.number])


# Correlation Matrix Heatmap
correlation_matrix = numeric_df.corr()
fig = ff.create_annotated_heatmap(z=correlation_matrix.values, x=list(correlation_matrix.columns), y=list(correlation_matrix.index), annotation_text=correlation_matrix.round(2).values, showscale=True, colorscale='Viridis')
fig.update_layout(title='Correlation Matrix Heatmap')
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the number of permits issued over time
plt.figure(figsize=(14, 7))
df['AppliedDate'].groupby(filtered_df['AppliedDate'].dt.to_period('M')).count().plot(kind='line')
plt.title('Number of Permits Applied Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Permits')
plt.show()


In [ ]:
# Additional Feature Engineering
df['Month'] = df['AppliedDate'].dt.month
df['Year'] = df['AppliedDate'].dt.year
df['DayOfWeek'] = df['AppliedDate'].dt.dayofweek
df['Quarter'] = df['AppliedDate'].dt.quarter


In [ ]:
# Aggregate data by contractor
contractor_performance = filtered_df.groupby('ContractorCompanyName').agg({
    'EstProjectCost': ['mean', 'median', 'sum'],
    'ApplicationToIssueTime': 'mean',
    'IssueToCompletionTime': 'mean'
}).reset_index()

# Flatten the MultiIndex columns
contractor_performance.columns = ['ContractorCompanyName', 'AvgProjectCost', 'MedianProjectCost', 'TotalProjectCost', 'AvgApplicationToIssueTime', 'AvgIssueToCompletionTime']


In [ ]:
# Top 10 contractors by average project cost
top_contractors = contractor_performance.sort_values(by='AvgProjectCost', ascending=False).head(10)

# Visualize the top contractors
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 7))
plt.barh(top_contractors['ContractorCompanyName'], top_contractors['AvgProjectCost'])
plt.xlabel('Average Project Cost')
plt.ylabel('Contractor Company Name')
plt.title('Top 10 Contractors by Average Project Cost')
plt.gca().invert_yaxis()
plt.show()


# Trend Analysis

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Decompose the time series
decomposition = seasonal_decompose(monthly_permits, model='additive')

# Plot the decomposed components
plt.figure(figsize=(14, 10))

plt.subplot(411)
plt.plot(decomposition.observed, label='Observed')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(decomposition.trend, label='Trend')
plt.legend(loc='best')

plt.subplot(413)
plt.plot(decomposition.seasonal, label='Seasonal')
plt.legend(loc='best')

plt.subplot(414)
plt.plot(decomposition.resid, label='Residual')
plt.legend(loc='best')

plt.tight_layout()
plt.show()


In [ ]:
# Analyze the decomposed components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Extract seasonal patterns
seasonal_pattern = seasonal.groupby(seasonal.index.month).mean()

# Plot the seasonal pattern
plt.figure(figsize=(14, 7))
seasonal_pattern.plot(marker='o')
plt.title('Average Seasonal Pattern')
plt.xlabel('Month')
plt.ylabel('Seasonal Effect')
plt.grid(True)
plt.show()

# Recommendations
# - Identify peak months for permit issuance
peak_months = seasonal_pattern.nlargest(3)
low_months = seasonal_pattern.nsmallest(3)

print("Peak Months for Permit Issuance:")
print(peak_months)

print("\nLow Months for Permit Issuance:")
print(low_months)


# Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming `filtered_df` is your DataFrame and already has the necessary features
features = [
    'ApplicationToIssueTime', 'IssueToCompletionTime', 'TotalPermitTime', 
    'AppliedYear', 'AppliedMonth', 'AppliedDay'
]

X = filtered_df[features]

# Define the target variable (if predicting time-based data, modify accordingly)
# For this example, we need to set a target variable. Let's assume you're predicting `TotalPermitTime`
y = filtered_df['TotalPermitTime']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)
model_xgb.fit(X_train, y_train)

# Make predictions
y_pred_xgb = model_xgb.predict(X_test)

# Evaluate the model
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f'XGBoost MAE: {mae_xgb}')
print(f'XGBoost MSE: {mse_xgb}')
print(f'XGBoost RMSE: {rmse_xgb}')
print(f'XGBoost R-squared: {r2_xgb}')


In [ ]:
# plot


# For XGBoost
y_pred_xgb = model_xgb.predict(X_test)


import matplotlib.pyplot as plt

# Plot actual vs. predicted values
plt.figure(figsize=(14, 7))

# XGBoost predictions
plt.subplot(2, 1, 1)
plt.plot(y_test.values, label='Actual Values', color='blue')
plt.plot(y_pred_xgb, label='XGBoost Predictions', color='red', linestyle='--')
plt.title('XGBoost Model Predictions vs Actual Values')
plt.xlabel('Sample Index')
plt.ylabel('Target Value')
plt.legend()

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_mae = cross_val_score(model_xgb, X_scaled, y, cv=5, scoring='neg_mean_absolute_error')
cv_mse = cross_val_score(model_xgb, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
cv_r2 = cross_val_score(model_xgb, X_scaled, y, cv=5, scoring='r2')

print(f'Cross-Validated MAE: {-cv_mae.mean()}')
print(f'Cross-Validated MSE: {-cv_mse.mean()}')
print(f'Cross-Validated R-squared: {cv_r2.mean()}')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

residuals = y_test - y_pred_xgb

# Plotting residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.show()

# Residuals vs. Predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_xgb, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Value')
plt.ylabel('Residual')
plt.show()


In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    model_xgb, X_scaled, y, cv=5, scoring='neg_mean_squared_error', 
    train_sizes=np.linspace(0.1, 1.0, 10)
)

train_scores_mean = -train_scores.mean(axis=1)
test_scores_mean = -test_scores.mean(axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training error")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Validation error")
plt.title('Learning Curve')
plt.xlabel('Training examples')
plt.ylabel('MSE')
plt.legend(loc="best")
plt.show()


In [ ]:
importances = model_xgb.feature_importances_
feature_names = filtered_df[features].columns

plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances)
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()
